In [1]:
include("../src/Julia.jl")

objc[56485]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[56485]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[56485]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[56485]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


pcgWorker (generic function with 1 method)

In [31]:
gname = "../../graphs/randdense_iu_1000_1/"

"../../graphs/randdense_iu_1000_1/"

In [32]:
a = readFromFile(gname * "graph.mtx");

In [33]:
tree = []
str = []
data = []

for i in 1:3
    curTree = readFromFile(gname * "tree$(i).mtx");
    rest = a - curTree
    curStr = compStretches(curTree,rest).nzval
    
    push!(tree, curTree)
    push!(str, curStr)
    push!(data, (sum(curStr)/2, mean(curStr)))
end

In [34]:
# check these values when editing the description
n = a.n;
iu = true
p = 0.25;
wmax = 1000;

In [35]:
function strDist{Tv,Ti}(tr::SparseMatrixCSC{Tv,Ti}, a::SparseMatrixCSC{Tv,Ti})
    
    res = Array{ASCIIString,1}(0)
    
    str = compStretches(tr, a-tr).nzval;
    
    vmax = maximum(str)
    inter = (0,1)
    while inter[2] < vmax
        cnt = 0
        for i in 1:length(str)
            if inter[1] <= str[i] && str[i] < inter[2]
                cnt = cnt + 1
            end
        end
        # don't count edges twice
        cnt = ceil(Int64,cnt / 2)
        
        push!(res, "[$(inter[1]),$(inter[2])) - $(cnt)")
        
        inter = (inter[2], inter[2] * 2)
    end
    
    push!(res, "")
    
    return res;
    
end

strDist (generic function with 1 method)

In [36]:
# Return the sorted eigenvalues of lap(a) * lap(tr)^-1
function getEigs{Tv,Ti}(tr::SparseMatrixCSC{Tv,Ti}, a::SparseMatrixCSC{Tv,Ti})
    
    la = lap(a);
    ts = full(tr);
    eigVals = real(eigs(la * pinv(lap(ts)), nev=a.n-2)[1])
    
    eigsString = ""
    for i in length(eigVals):-1:1
        eigsString = eigsString * string(round(eigVals[i],4)) * " "
    end
    
    return eigsString
    
end

getEigs (generic function with 1 method)

In [37]:
output = []
push!(output, "A random graph with $(n) vertices and $(ceil(Int64,nnz(a)/2)) edges.")
push!(output, "")
push!(output, "The graph is generated in the following way:")
push!(output, "    - for each vertex, select n / $(1/p) random neighbors")
push!(output, "    - get rid of duplicate edges")
if iu
    push!(output, "    - weights are reals sampled uniformly from 1/[1..$(wmax)]")
else
    push!(output, "    - weights are integers sampled uniformly from [1..$(wmax)]")
end
push!(output, "")
push!(output, "There are three trees in the folder.")
push!(output, "")

for i in 1:3
    push!(output, "Tree$(i) - total stretch = $(data[i][1]), ave stretch = $(data[i][2])")
    @time append!(output, strDist(tree[i],a))
    push!(output, "Eigenvalues of lap(a) * lap(tree$(i))^-1")
    @time push!(output, getEigs(tree[i],a))
    push!(output, "")
    push!(output, "")
end

desc = open(gname * "description2.txt", "w")
for i in 1:length(output)
    println(desc, output[i])
end
close(desc)

  0.036981 seconds (10.11 k allocations: 19.047 MB, 9.68% gc time)
 14.891178 seconds (36.04 k allocations: 206.792 MB, 0.56% gc time)
  0.034215 seconds (276 allocations: 18.585 MB, 20.51% gc time)
 13.954303 seconds (32.76 k allocations: 206.878 MB, 0.60% gc time)
  0.035209 seconds (292 allocations: 18.585 MB, 22.52% gc time)
 13.716271 seconds (32.74 k allocations: 206.850 MB, 0.21% gc time)
